In [3]:
import pandas as pd
import datetime
import dateparser
import numpy as np
from datetime import date

### Nº de Baixas Por Isolamento

In [22]:
def baixas_all_data():
    df_baixas = pd.read_csv('original_files/Baixas por Isolamento.csv')
    #Select the relevant information, deleting the first 8 rows
    df_baixas = df_baixas.iloc[6:]
    baixas = df_baixas[df_baixas.columns[:3]]
    #Reset indexes
    baixas.reset_index(drop=True, inplace=True)
    #Get indexes that separate tables
    b = np.concatenate(( [True], pd.isnull(baixas.iloc[:,0]), [True] )) 
    indexes = np.flatnonzero(b[1:] != b[:-1]) 
    baixas_all = baixas[indexes[0]:indexes[1]]
    #dropping rows with nan values
    baixas_all.dropna(inplace=True)
    #renaming the columns
    baixas_all.columns = ['DATA', 'POR DIA', 'ACUMULADOS']
    #fomatting the date 
    baixas_all['DATA'] = baixas_all['DATA'].apply(dateparser.parse)
    baixas_all.to_csv(r'dataframes/baixas_all.csv', index = False)
    return baixas_all

In [23]:
baixas_all_data().tail()

C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,DATA,POR DIA,ACUMULADOS
124,2020-07-03,461,136144
125,2020-07-04,21,136165
126,2020-07-05,8,136173
127,2020-07-06,486,136659
128,2020-07-07,222,136881


In [24]:
def baixas_distrito_data():
    df_baixas = pd.read_csv('original_files/Baixas por Isolamento.csv')
    #Select the relevant information, deleting the first 8 rows
    df_baixas = df_baixas.iloc[6:]
    baixas = df_baixas[df_baixas.columns[:3]]
    #Reset indexes
    baixas.reset_index(drop=True, inplace=True)
    #Get indexes that separate tables
    b = np.concatenate(( [True], pd.isnull(baixas.iloc[:,0]), [True] )) 
    indexes = np.flatnonzero(b[1:] != b[:-1]) 
    baixas_distrito = baixas[indexes[4]:indexes[5]]
    #dropping rows with nan values
    baixas_distrito.dropna(axis=1, how='any', inplace=True)
    #renaming the columnsS']
    baixas_distrito.columns = ['DISTRITO', 'TOTAL']
    baixas_distrito.to_csv(r'dataframes/baixas_distrito.csv', index = False)
    return baixas_distrito

In [26]:
baixas_distrito_data().head()

C:\Users\Ana\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,DISTRITO,TOTAL
135,TOTAL,136881
136,AVEIRO,69214
137,BEJA,480
138,BRAGA,7998
139,BRAGANÇA,535


### Lay Off - Estimativa

In [27]:
def layoff_data():
    df_layoff = pd.read_csv('original_files/Layoff – Estimativa .csv')
    df_layoff.dropna(axis=0, how='any', inplace=True)
    df_layoff.columns = ['DATA', 'Nº NISS_EE', 'Nº TRABALHADORES', 
                            'REMUNERAÇÕES DECLARADAS']
    df_layoff.drop(['Nº NISS_EE'], axis=1, inplace=True) 
    df_layoff['DATA'] = df_layoff['DATA'].apply(lambda x: x.split(' ')[0])
    df_layoff['DATA'] = df_layoff['DATA'].apply(lambda x: x.replace('/', '-'))
    df_layoff['DATA'] = df_layoff['DATA'].apply(lambda x: x.split('-'))
    df_layoff['month'] = df_layoff['DATA'].apply(lambda x: x[1])
    df_layoff['day'] = df_layoff['DATA'].apply(lambda x: x[2] if len(x[0]) > 2 else x[0])
    df_layoff['year'] = df_layoff['DATA'].apply(lambda x: x[0] if len(x[0]) > 2 else x[2])
    df_layoff['DATA'] = pd.to_datetime(df_layoff[['day', 'month', 'year']])
    df_layoff.drop(['month', 'day', 'year'], axis=1, inplace=True) 
    df_layoff['DATA'] = df_layoff['DATA'].mask(df_layoff['DATA'].dt.year == 2021, 
                             df_layoff['DATA'] + pd.offsets.DateOffset(year=2020))
    df_layoff.to_csv(r'dataframes/df_layoff.csv', index = False)
    return df_layoff

In [28]:
layoff_data().head()

C:\Users\Ana\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:692: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


,DATA,Nº TRABALHADORES,REMUNERAÇÕES DECLARADAS
6,2020-03-31,72507,69174523
7,2020-04-01,231683,225500911
8,2020-04-02,350028,339900463
9,2020-04-03,425287,434790415
10,2020-04-04,551955,570960157


### Layoff – Estim. - CAE,Dim,Dist

#### DATA RELEVANT TO WORK SECTORS-Amount of companies on layoff

In [4]:
def historic_sector_data():
    data1 = pd.read_csv('original_files/historical_data_company.csv')
    data2 = pd.read_csv('original_files/Layoff – Estim. - CAE,Dim,Dist.csv')

In [7]:
#Select the relevant information, deleting the first 8 rows
df=data2.iloc[8:]

#import today´s date
today=date.today()

#Select the columns that are relevant to work sectors
df_work=df[df.columns[0:3]]
df_work=df_work.iloc[2:23]

#Rename columns
df_work=df_work.rename(columns={'EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado':'Index','Unnamed: 1':'Setor',
                       'Unnamed: 2':today,'Unnamed: 3':'Nº TRABALHADORES','Unnamed: 4':'Feminino','Unnamed: 5':'Masculino'})

#Drop column
df_work=df_work.drop(['Index'],axis=1)
#Find total value for last report
#total=sum(df_work[today])
#total

#add new total row to new data
#new_row=pd.Series(data={'Setor':'TOTAL',today:total})
#df_work=df_work.append(new_row, ignore_index=True)

In [9]:
today

datetime.date(2020, 7, 13)

In [4]:
#merge historical data with new data
new_historical=pd.merge(left=data1, right=df_work, left_on='Setor', right_on='Setor')
new_historical=new_historical.sort_values(by=[today],ascending=False)
new_historical

,Unnamed: 0,Setor,2020-04-17,2020-04-20,2020-04-23,2020-04-24,2020-04-27,2020-04-29,2020-04-30,2020-05-04,...,2020-05-13,2020-05-14,2020-05-15,2020-05-19,2020-05-21,2020-05-27,2020-06-03,2020-06-09,2020-06-16,2020-06-27
1,A,"Agricultura, produção animal, caça, floresta e...",581,614,660,675,687,712,728,755,...,818,824,827,833,839,848,860,864,865,871
17,Q,Actividades de saúde humana e apoio social,6328,6534,6870,6962,7063,7180,7258,7428,...,7703,7727,7741,7767,7789,7831,7867,7922,7935,7947
8,H,Transportes e armazenagem,4137,4357,4734,4859,5001,5203,5322,5612,...,6084,6121,6153,6231,6288,6362,6458,6519,6560,6591
2,B,Indústrias extractivas,42,45,47,47,47,51,51,53,...,59,59,59,63,63,63,63,64,64,64
19,S,Outras actividades de serviços,4266,4433,4714,4776,4857,4975,5047,5182,...,5419,5439,5449,5475,5500,5535,5574,5601,5613,5624
0,s/CAE,PESSOAS SINGULARES COM TRABALHADORES A CARGO,4017,4182,4467,4568,4652,4777,4852,5003,...,5256,5279,5289,5316,5335,5368,5409,5435,5455,5464
13,M,"Actividades de consultoria, científicas, técni...",3434,3588,3887,3956,4044,4180,4256,4498,...,4966,5001,5037,5087,5119,5173,5249,5296,5324,5340
15,O,Administração pública e defesa; segurança soci...,24,25,30,32,32,33,35,36,...,43,45,45,46,46,46,47,47,47,47
18,R,"Actividades artísticas, de espectáculos, despo...",3562,3685,3877,3938,3994,4074,4131,4229,...,4396,4415,4424,4439,4455,4489,4527,4543,4557,4574
20,T + U,Actividades das famílias empregadoras de pesso...,3,3,3,3,3,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [5]:
new_historical.to_csv('original_files/historical_data_company.csv')

#### DATA RELEVANT TO WORK SECTORS - Amount of people on layoff

In [10]:
data3 = pd.read_csv('original_files/historical_data_person.csv')
data4 = pd.read_csv('original_files/Layoff – Estim. - CAE,Dim,Dist.csv')

In [11]:
#Select the relevant information, deleting the first 8 rows
df=data2.iloc[8:]

#import today´s date
today=date.today()

#Select the columns that are relevant to work sectors
df_work=df[df.columns[0:4]]
df_work=df_work.iloc[2:23]

#Rename columns
df_work=df_work.rename(columns={'EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado':'Index','Unnamed: 1':'Setor',
                       'Unnamed: 2':'Nº NISS_EE','Unnamed: 3':today,'Unnamed: 4':'Feminino','Unnamed: 5':'Masculino'})

#Drop column
df_work=df_work.drop(['Index','Nº NISS_EE'],axis=1)

#Add Total Column
total=sum(df_work[today])
total

#add new total row
new_row=pd.Series(data={'Setor':'TOTAL',today:total})
df_work=df_work.append(new_row, ignore_index=True)

df_work

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [8]:
#merge historical data with new data
new_historical = pd.merge(left=data3, right=df_work, left_on='Setor', right_on='Setor')
new_historical=new_historical.sort_values(by=[today],ascending=False)
new_historical

,Unnamed: 0,Unnamed: 0.1,Setor,2020-04-17,2020-04-20,2020-04-23,2020-04-24,2020-04-27,2020-04-29,2020-04-30,...,2020-05-13,2020-05-14,2020-05-15,2020-05-19,2020-05-21,2020-05-27,2020-06-03,2020-06-09,2020-06-16,2020-06-29
17,17,Q,Actividades de saúde humana e apoio social,70938,75602,80920,82671,84392,85851,86961,...,91816,92126,92542,92842,93392,93878,94573,95543,95651,95926
1,1,A,"Agricultura, produção animal, caça, floresta e...",5252,5521,5824,5913,5964,6068,6126,...,6628,6688,6697,6724,6923,6947,7007,7016,7019,7035
8,8,H,Transportes e armazenagem,56195,57252,60642,61409,61779,62452,63109,...,66238,66443,66615,66973,67158,67450,67764,67987,68085,68152
4,4,D,"Electricidade, gás, vapor, água quente e fria ...",564,564,571,579,580,580,580,...,644,644,644,644,644,644,644,644,644,644
6,6,F,Construção,39280,40560,43216,44928,45515,46482,47162,...,52210,52376,52440,52701,52834,53263,53608,53956,53981,54088
11,11,K,Actividades financeiras e de seguros,3387,3433,3966,3979,3982,4094,4154,...,5100,5110,5112,5131,5187,5195,5225,5253,5255,5268
13,13,M,"Actividades de consultoria, científicas, técni...",30729,31791,33727,34046,34368,35066,35357,...,48646,48958,49095,49337,49514,49805,50219,50446,50538,50600
5,5,E,"Captação, tratamento e distribuição de água; s...",1371,1525,1620,1623,1638,1711,1715,...,3856,3944,3944,3955,3957,3963,3988,4016,4016,4019
16,16,P,Educação,25496,27337,29184,29654,30203,31974,32248,...,34456,34482,34568,34600,34687,34770,34876,34905,34995,35070
19,19,S,Outras actividades de serviços,26635,27287,29039,29504,29977,30561,31357,...,33253,33382,33359,33439,33553,33674,33955,34074,34131,34160


In [9]:
new_historical.to_csv('original_files/historical_data_person.csv')

#### DATA RELEVANT TO ORGANIZATION DIMENSION

In [18]:
def organization_dimension():
    data5 = pd.read_csv('original_files\historical_data_company_size.csv')
    data5=data5.drop(['Unnamed: 0'],axis=1)
    data6 = pd.read_csv('original_files/Layoff – Estim. - CAE,Dim,Dist.csv')
    #Create new dataset
    df=data6 
    # Search for row and column position of desired data
    i, c = np.where(df == 'até 10 trabalhadores')
    e, d=np.where(df=='>= 250 trabalhadores')
    #Create names for data range, for easier identification
    row_size=i+2
    col1=(c+1)
    col2=(d+1)
    #Splice the data with the selected rows found on the previous step
    df2=df.iloc[int(i):int(row_size),int(c):int(col2)]
    #perform some data cleaning
    df2=df2.reset_index()
    df2=df2.drop('index',1)
    df2.columns=df2.iloc[0]
    df2=df2.drop([0])
    #transpose the data to match the axis of the historical data
    df2=df2.transpose()
    df2=df2.reset_index()
    #import today´s date
    today=date.today()
    #Rearrange column names
    df2.columns=['TOTAL',today]
    new_historical = pd.merge(left=data5, right=df2, left_on='TOTAL', right_on='TOTAL')
    new_historical=new_historical.sort_values(by=[today],ascending=False)
    new_historical.to_csv('original_files/historical_data_company_size.csv')
    return new_historical

In [19]:
organization_dimension().head()

C:\Users\Ana\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,TOTAL,2020-04-17,2020-04-20,2020-04-23,2020-04-24,2020-04-27,2020-04-29,2020-04-30,2020-05-04,2020-05-05,...,2020-05-21,2020-05-27,2020-06-03,2020-06-09,2020-06-16,2020-06-23,2020-07-01,2020-07-08,2020-07-13,2020-07-13
0,até 10 trabalhadores,66380,69160,74152,75624,77125,79402,80675,83466,84449,...,90444,91355,92222,92619,92884,93068,93363,93469,93469,93469
1,>= 250 trabalhadores,435,454,473,479,484,494,496,514,520,...,538,533,543,541,541,539,543,542,542,542
2,26 a 49 trabalhadores,3315,3443,3661,3706,3756,3825,3855,3975,4026,...,4238,4215,4236,4253,4267,4261,4274,4305,4305,4305
3,50 a 249 trabalhadores,2612,2706,2886,2915,2960,3001,3036,3159,3172,...,3266,3248,3264,3300,3305,3318,3337,3357,3357,3357
4,11 a 25 trabalhadores,10065,10438,11102,11269,11432,11694,11811,12137,12268,...,12885,12889,12967,13108,13122,13183,13243,13290,13290,13290


#### DATA RELEVANT TO REGION

In [20]:
def region_data():
    #Select new dataset
    df=data4
    #Drop all set of row and columns that are filled with NaN
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    #Delete the first rows that don't present any important information
    df=df.iloc[9:]
    #Search for the relevant information that presents the region information, and store the column number
    lower=df.columns.get_loc("EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado.1")
    #Select the upper range of columns with relevant data
    upper=lower+3
    #filter the data with the selected parameters
    df_region=df.iloc[:,lower:upper]
    #rename columns and do some data cleaning
    df_region=df_region.rename(columns={'EEs QUE ENTREGARAM DOCUMENTO - COVID19 - Layoff Simplificado.1':'Region',
                                    'Unnamed: 8':'N° Empresas',
                       'Unnamed: 9':'Percentual(%)'})
    df_region['Region']=df_region['Region'].replace({'AVEIRO':'Aveiro','BEJA':'Beja','BRAGA':'Braga','BRAGANÇA':'Bragança',
                                                 'CASTELO BRANCO':'Castelo Branco','COIMBRA':'Coimbra','ÉVORA':'Évora',
                                                 'FARO':'Faro','GUARDA':'Guarda','LEIRIA':'Leiria','LISBOA':'Lisboa',
                                                 'PORTALEGRE':'Portalegre','PORTO':'Porto','SANTARÉM':'Santarém',
                                                 'SETÚBAL':'Setúbal','VIANA DO CASTELO':'Viana do Castelo',
                                                 'VILA REAL':'Vila Real','VISEU':'Viseu','R.A.AÇORES':'R.A.Açores',
                                                 'R.A.MADEIRA':'R.A.Madeira'})
    df_region['Percentual(%)']=df_region['Percentual(%)']*100
    df_region=df_region.reset_index()
    df_region=df_region.drop('index',1)
    df_region.dropna(axis=0, how='all', inplace=True)
    df_region.to_csv(r'dataframes/organization_region.csv', index = False)
    return df_region

In [21]:
region_data().head()

,Region,N° Empresas,Percentual(%)
0,Aveiro,7561,6.576899
1,Beja,1014,0.882023
2,Braga,10969,9.541331
3,Bragança,1255,1.091656
4,Castelo Branco,1507,1.310857


### Redução de Actividade TI e MOE

#### Redução de atividade por Dia

In [49]:
def reducao_atividade_byday():
    
    reducao_atividade=pd.read_csv('original_files/Redução de Actividade TI e MOE.csv')
    # Drop rows with no info
    reducao_atividade.dropna(how='all',inplace=True)
    # Reset indexes
    reducao_atividade.reset_index(drop=True, inplace=True)
    # Get indexes that separate tables
    m = np.concatenate(( [True], reducao_atividade.iloc[:,1:13].isna().all(axis=1), [True] )) 
    indexes = np.flatnonzero(m[1:] != m[:-1])
    # Separate tables
    red_byday = reducao_atividade[indexes[0]+2:indexes[1]]
    
    # Clean byday table 
    red_byday=red_byday.dropna(axis=1,how='all')
    red_byday.columns = ['DATA', 'TI_ParagemTotal', 'TI_Reducao','TI_Total', 'PRO_TI_ParagemTotal', 'PRO_TI_Reducao','PRO_TI_Total', 'MOE_ParagemTotal', 'MOE_Reducao','MOE_Total', 'PRO_MOE_ParagemTotal', 'PRO_MOE_Reducao','PRO_MOE_Total']

    # Formatting the date - sometimes some values are already of data type
#     date_inds=pd.notnull(pd.to_datetime(red_byday['DATA'],errors='coerce'))
#     if date_inds.index[date_inds].size > 0:
#         red_byday['DATA'][date_inds]=red_byday['DATA'][date_inds].dt.strftime('%d/%b')
    red_byday['DATA'] = red_byday['DATA'].apply(dateparser.parse,date_formats=['%d/%b'])

    # Remove columns that dont have datetype in DATA, for example columns with the TOTAL
    red_byday.dropna(subset=['DATA'],inplace=True)
    # Reset indexes
    red_byday.reset_index(drop=True, inplace=True)

    # Save csv
    red_byday.to_csv(r'dataframes/reducao_atividade_porDia.csv', index = False)

#### Redução de atividade por Distrito

In [45]:
def reducao_atividade_bydistrict():
    
    reducao_atividade=pd.read_csv('original_files/Redução de Actividade TI e MOE.csv')
    # Drop rows with no info
    reducao_atividade.dropna(how='all',inplace=True)
    # Reset indexes
    reducao_atividade.reset_index(drop=True, inplace=True)
    # Get indexes that separate tables
    m = np.concatenate(( [True], reducao_atividade.iloc[:,1:13].isna().all(axis=1), [True] )) 
    indexes = np.flatnonzero(m[1:] != m[:-1])
    # Separate tables
    red_bydistrict = reducao_atividade[indexes[2]:indexes[3]]
    
    red_bydistrict_months=red_bydistrict
    # Clean bydistrict table 
    red_bydistrict_months.reset_index(drop=True, inplace=True)
    red_bydistrict_months=red_bydistrict_months.set_index(red_bydistrict_months.columns.values[0])
    red_bydistrict_months.index.name='Distritos'
    # Find columns corresponding to the total counts
    checknull_months=red_bydistrict_months.iloc[1].notnull() 
    checktotal_months=red_bydistrict_months.iloc[1].str.lower()!='total'
    checknull_title=red_bydistrict_months.iloc[0].notnull()
    # Generate multi level columns
    col_indexes=np.nonzero(checknull_title.array)
    col_indexes=col_indexes[0]
    columns=[]
    for j in range(4):
        if j<3: 
            max_range=col_indexes[j+1]
        else:
            max_range=red_bydistrict_months.shape[1]
        for i in range(col_indexes[j],max_range):
            columns.append((red_bydistrict_months.iloc[0,col_indexes[j]],red_bydistrict_months.iloc[1,i]))
    red_bydistrict_months.columns = pd.MultiIndex.from_tuples(columns,names=['Type', 'Month'])
    # Get columns concerning each month
    red_bydistrict_months=red_bydistrict_months.iloc[2:, checknull_months.array & checktotal_months.array]
    
    # Create dataframe with total counts for each district
    red_bydistrict_months=red_bydistrict_months.astype('int64')
    red_bydistrict_total=red_bydistrict_months.groupby(level=['Type'],axis=1).sum()
    red_bydistrict_total.columns=['PRO_MOE','PRO_TI','MOE','TI']

    # Save csvs (per month and total)
    red_bydistrict_months.to_csv(r'dataframes/reducao_atividade_porDistrito_porMes.csv')
    red_bydistrict_total.to_csv(r'dataframes/reducao_atividade_porDistrito_Total.csv')
    
    ## APPEND TO HISTORICAL DATA
    # Transform columns into indexes
    df_aux=red_bydistrict_total.stack().to_frame()
    # Change columns to the file's date
    df_aux.columns=[date.today()]

    # Import dataframe with historical data
    red_bydistrict_historical=pd.read_csv('dataframes/reducao_atividade_bydistrict_historicalData.csv',index_col=[0,1])
    # Append to dataframe with historical data for reducao de atividade por distrito
    red_bydistrict_historical=pd.concat([red_bydistrict_historical,df_aux],axis=1,join='outer')
    red_bydistrict_historical.to_csv('dataframes/reducao_atividade_bydistrict_historicalData.csv')

#### Redução de atividade por Sexo

In [46]:
def reducao_atividade_bysex():
    
    reducao_atividade=pd.read_csv('original_files/Redução de Actividade TI e MOE.csv')
    # Drop rows with no info
    reducao_atividade.dropna(how='all',inplace=True)
    # Reset indexes
    reducao_atividade.reset_index(drop=True, inplace=True)
    # Get indexes that separate tables
    m = np.concatenate(( [True], reducao_atividade.iloc[:,1:13].isna().all(axis=1), [True] )) 
    indexes = np.flatnonzero(m[1:] != m[:-1])
    # Separate tables
    red_bysex = reducao_atividade[indexes[4]+2:indexes[5]]
    
    # Clean bysex table 
    red_bysex=red_bysex.dropna(axis=1)
    red_bysex.columns=['TipoPedido','Total','Feminino','Masculino']
    if len(red_bysex.iloc[:,0])==3:
        red_bysex.iloc[:,0]=['Trabalhador Independente','Prorrogação Trabalhador Independente','Membro Orgão Estatutário']
    elif len(red_bysex.iloc[:,0])==4:
        red_bysex.iloc[:,0]=['Trabalhador Independente','Prorrogação Trabalhador Independente','Membro Orgão Estatutário','Prorrogação Membro Orgão Estatutário']
    red_bysex.reset_index(drop=True, inplace=True)

    # Save csv
    red_bysex.to_csv(r'dataframes/reducao_atividade_porSexo.csv', index = False)
    
    ## APPEND TO HISTORICAL DATA
    # Transform columns into indexes
    red_bysex_new=red_bysex.set_index('TipoPedido').stack().to_frame()
    # Change columns to the file's date
    red_bysex_new.columns=[date.today()]

    # Import dataframe with historical data
    red_bysex_historical=pd.read_csv('dataframes/reducao_atividade_bysex_historicalData.csv',index_col=[0,1])
    # # Append to dataframe with historical data for reducao de atividade por sexo
    red_bysex_historical=pd.concat([red_bysex_historical,red_bysex_new],axis=1,join='outer')
    red_bysex_historical.to_csv('dataframes/reducao_atividade_bysex_historicalData.csv')

### Despedimentos coletivos

In [47]:
def despedimentos_coletivos():
    despedimentos=pd.read_csv('original_files/Despedimentos coletivos.csv')
    # Drop first column which is empty
    despedimentos=despedimentos.drop(despedimentos.columns[0],axis=1)
    # Drop rows with nan
    despedimentos.dropna(inplace=True)
    despedimentos.reset_index(drop=True, inplace=True)
    # Rename columns
    despedimentos.columns = ['DATA', 'COLETIVOS_TOTAL', 'COLETIVOS_MICRO','TRABALHADORES_TOTAL', 'TRABALHADORES_MICRO']
    # Save csv
    despedimentos.to_csv(r'dataframes/despedimentos_coletivos.csv', index = False)